In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -q transformers flax
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git  # VQGAN model in JAX
!pip install -q git+https://github.com/borisdayma/dalle-mini.git  # Model files

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 207 kB 33.1 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 126 kB 47.1 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 235 kB 4.3 MB/s 
     |████████████████████████████████| 64 kB 2.5 MB/s 


In [ ]:
!pip install flax==0.3.5

     |████████████████████████████████| 193 kB 4.2 MB/s 
  Attempting uninstall: flax
    Found existing installation: flax 0.3.6
    Uninstalling flax-0.3.6:
      Successfully uninstalled flax-0.3.6


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-jvobzo2c
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-jvobzo2c
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369106 sha256=a6686c09b5da1f1707cd23fd901f39ff33baf352423bfd42d8834f22ee2535bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-69lt9ok_/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from transformers import BartTokenizer
import jax
import random
from tqdm.notebook import tqdm, trange

from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image

from transformers import CLIPProcessor, FlaxCLIPModel, CLIPModel

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.model_zoo as model_zoo
from torchvision.transforms import ToPILImage
from torchvision.transforms import Compose, Resize, InterpolationMode, Normalize
import torch
import clip
from PIL import Image

import easydict
import cv2

import os
from glob import glob
import pickle
import random
import math

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

In [ ]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth'
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, in_channels=12):
        self.inplanes = 16
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=7, stride=1, padding=3,
                               bias=False) # ori : stride = 2
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(1, stride=1)
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

In [ ]:
# dataloader
class IMG_Dataset(Dataset):
    def __init__(self, args, is_validated=False):
        self.args = args
        if is_validated == True:
          self.args.mode = 'val'

        data_path = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/data', args.mode) + '.pkl'
        prompts_path = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/data', args.mode) + '_prompts.pkl'

        if os.path.isfile(data_path):
            with open(data_path, 'rb') as f:
                self.data = pickle.load(f)
                print('data num: ', len(self.data))
        else:
            self.data = list()
            images = list()
            prompts = list()
            prompt2idx = dict()
            idx2prompt = dict()

            texts_path = glob(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/corpus', args.mode)+'/*')
            for text in texts_path:
              i = 0
              images_path = glob(text + '/*')
              temp = []
              for image in images_path:
                i+=1
                img = cv2.imread(image)
                img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
                temp.append(img)
                if i == 2:
                  break
                
              prompt = images_path[0].split('/')[-2].replace('_', ' ')
              print(prompt)
              prompts.append(prompt)

              temp = np.array(temp)
              images.append(temp)
            
            prompt2idx = {prompt:i for i,prompt in enumerate(sorted(prompts))}
            idx2prompt = {prompt:i for prompt,i in enumerate(sorted(prompts))}
            prompts = [prompt2idx[prompt] for prompt in prompts]

            for image, prompt in zip(images, prompts):
              self.data.append((np.array([image, prompt]), 0))

            print(len(self.data))
            print(self.data[0][0].shape)

            with open(prompts_path, 'wb') as f:
                pickle.dump(idx2prompt, f)
            with open(data_path, 'wb') as f:
                pickle.dump(self.data, f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        x = self.data[i][0][0]
        prompt = self.data[i][0][1]
        y = self.data[i][1]
        return x, prompt, y

class IMG_Loader(object):
    def __init__(self, args, is_validated=False):
        super().__init__()

        dataset = IMG_Dataset(args)

        if args.mode == 'train':
            self.data_loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
        else:
            self.data_loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size)
        self.data_iter = self.data_loader.__iter__()
    
    def next_batch(self):
        try:
            batch = self.data_iter.__next__()
        except StopIteration:
            self.data_iter = self.data_loader.__iter__()
            batch = self.data_iter.__next__()
        
        return batch

In [ ]:
# trainer
class Trainer():
    def __init__(self, args):
        self.args = args
        self.loader = IMG_Loader(args)
        #self.model = MLP(args)
        self.model = CNN(args)
        #self.model = background_resnet(embedding_size=256)
        if self.args.device == 'cuda':
          self.model.cuda()

        self.criterion = nn.CrossEntropyLoss()
        self.optim = torch.optim.Adam(self.model.params_to_update)
        
    def train(self):
        # train
        total_step = len(self.loader.data_loader)
        #val_loader = IMG_Loader(self.args, is_validated=True)
        opt_epoch = 1
        min_val_loss = 1e9

        print(total_step)
        for epoch in range(self.args.epoch):
            self.model.train()
            total_loss = 0

            for i in range(len(self.loader.data_loader)):
                feature, prompt, label = self.loader.next_batch()
                feature = torch.tensor(feature).to(device=self.args.device, dtype=torch.float)
                label = torch.tensor(label).to(device=self.args.device)

                pred = self.model(feature, prompt)
                loss = self.criterion(pred, label)
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()

                total_loss += loss.item()
                if (i + 1) % 5 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, self.args.epoch, i + 1, total_step, loss.item()))

            #if (epoch+1)%50 == 0:
            '''
            val_loss = self.validate(val_loader)
            print ('Validation Loss: {:.20f}'.format(val_loss))
            if val_loss < min_val_loss:
              val_loss = min_val_loss
              opt_epoch = epoch+1
            '''
            torch.save(self.model.state_dict(), '/content/gdrive/MyDrive/Colab Notebooks/model_saved/checkpoint_'+str(epoch+1)+'.pt')

    def validate(self, val_loader=None):
        if val_loader is None:
          val_loader = IMG_Loader(self.args, is_validated=True)
        total_loss = 0
        total_step = len(val_loader.data_loader)

        self.model.load_state_dict(torch.load('/content/gdrive/MyDrive/Colab Notebooks/model_saved/checkpoint_2.pt'))
        self.model.eval()
        with torch.no_grad():
            for i in range(len(val_loader.data_loader)):
                feature, prompt, label = self.loader.next_batch()
                feature = torch.tensor(feature).to(device=self.args.device, dtype=torch.float)
                label = torch.tensor(label).to(device=self.args.device)

                pred = self.model(feature, prompt)
                loss = self.criterion(pred, label)
                total_loss += loss.item()

                if (i + 1) % 5 == 0:
                    print ('Validation Step [{}/{}], Loss: {:.4f}'.format(i + 1, total_step, loss.item()))

        return total_loss/total_step


    def test(self):
        # test
        #self.model.load_state_dict(torch.load(self.args.ckpt))
        self.model.load_state_dict(torch.load('/content/gdrive/MyDrive/Colab Notebooks/model_saved/checkpoint_6.pt', map_location=torch.device('cpu')))
        self.model.eval()

        pred_list = list()
        label_list = list()

        with torch.no_grad():
            for i in range(len(self.loader.data_loader)):
                feature, label = self.loader.next_batch()
                feature = torch.tensor(feature).to(device=self.args.device, dtype=torch.float)
                label = torch.tensor(label).to(device=self.args.device)

                pred = self.model(feature)
                pred = F.softmax(pred, dim=1)
                pred = torch.argmax(pred, dim=1)
                
                pred_list.extend(pred.tolist())
                label_list.extend(label.tolist())

        acc = accuracy_score(label_list, pred_list)
        print(acc)

In [ ]:
class background_resnet(nn.Module):
    def __init__(self, embedding_size, backbone='resnet18'):
        super(background_resnet, self).__init__()
        self.args = args
        self.prompt_path = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/data', args.mode) + '_prompts.pkl'
        with open(self.prompt_path, 'rb') as f:
          self.idx2prompt = pickle.load(f)
        print(self.idx2prompt)

        self.my_preprocess = Compose([
                                Resize(224, interpolation=InterpolationMode.BICUBIC),
                                Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
                            ])
        self.model, _ = clip.load("ViT-B/32", device=self.args.device)

        self.backbone = backbone
        # copying modules from pretrained models
        if backbone == 'resnet50':
            self.pretrained = resnet50(pretrained=False)
        elif backbone == 'resnet101':
            self.pretrained = resnet101(pretrained=False)
        elif backbone == 'resnet152':
            self.pretrained = resnet152(pretrained=False)
        elif backbone == 'resnet18':
            self.pretrained = resnet18(pretrained=False)
        elif backbone == 'resnet34':
            self.pretrained = resnet34(pretrained=False)
        else:
            raise RuntimeError('unknown backbone: {}'.format(backbone))
            
        self.fc0 = nn.Linear(128, embedding_size)
        #self.bn0 = nn.BatchNorm1d(embedding_size)
        self.relu = nn.ReLU()
        self.last = nn.Linear(embedding_size, 150528)
        
        self.params_to_update = list(self.pretrained.parameters()) + list(self.fc0.parameters()) + list(self.last.parameters())

    def forward(self, x, prompts):
        prompts = prompts.tolist()
        prompts = [self.idx2prompt[prompt] for prompt in prompts]
        prompt = prompts[0]

        out = x.permute(0,1,4,2,3)
        out = out.reshape(1,-1,224,224)

        out = self.pretrained.conv1(out)
        out = self.pretrained.bn1(out)
        out = self.pretrained.relu(out)
        
        out = self.pretrained.layer1(out)
        out = self.pretrained.layer2(out)
        out = self.pretrained.layer3(out)
        out = self.pretrained.layer4(out)
        
        out = F.adaptive_avg_pool2d(out,1) # [batch, 128, 1, 1]
        out = torch.squeeze(out) # [batch, n_embed]
        # flatten the out so that the fully connected layer can be connected from here
        out = out.view(x.size(0), -1) # (n_batch, n_embed)
        spk_embedding = self.fc0(out)
        out = F.relu(spk_embedding) # [batch, n_embed]
        out = self.last(out)

        out = out.uniform_(0,1)*255
        out = out.reshape(x.size(0),1,224,224,3)

        if self.args.mode != 'train':
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'.png', (torch.squeeze(out)).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_0_.png', (torch.squeeze(x)[0]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_1_.png', (torch.squeeze(x)[1]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_2_.png', (torch.squeeze(x)[2]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_3_.png', (torch.squeeze(x)[3]).cpu().numpy())
        
        out = torch.squeeze(out)
        out = torch.unsqueeze(out, dim=0).permute(0,3,1,2)
        x = torch.squeeze(x).permute(0,3,1,2)

        # evaluate scores
        images = torch.cat([self.my_preprocess(out), self.my_preprocess(x)])
        text = torch.cat([clip.tokenize([prompt]).to(self.args.device)])
        logits_per_image, logits_per_text = self.model(images, text)

        logits = torch.squeeze(logits_per_image)
        logits = logits.reshape(1,-1)
        return logits

In [ ]:
# MLP
class MLP(nn.Module):
    def __init__(self, args):
        super(MLP, self).__init__()
        self.args = args
        self.prompt_path = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/data', args.mode) + '_prompts.pkl'

        with open(self.prompt_path, 'rb') as f:
          self.idx2prompt = pickle.load(f)
        print(self.idx2prompt)
        self.layer1 = nn.Linear(301056, 128)
        self.layer2 = nn.Linear(128, 150528)
        self.relu = nn.ReLU()

        self.my_preprocess = Compose([
                                Resize(224, interpolation=InterpolationMode.BICUBIC),
                                Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
                            ])
        self.model, _ = clip.load("ViT-B/32", device=self.args.device)
        # set up model and processor
        #self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        #self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", do_resize=False, do_center_crop=False)
        
        self.params_to_update = list(self.layer1.parameters()) + list(self.layer2.parameters())

    def forward(self, x, prompts):
        prompts = prompts.tolist()
        prompts = [self.idx2prompt[prompt] for prompt in prompts]
        prompt = prompts[0]

        print('x: ', x.shape)

        out = x.reshape(x.size(0), -1)
        out = self.layer1(out)
        out = self.relu(out)
        out = self.layer2(out)
        out = out.uniform_(0,1)*255
        out = out.reshape(x.size(0),1,224,224,3)

        if self.args.mode != 'train':
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'.png', (torch.squeeze(out)).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_0_.png', (torch.squeeze(x)[0]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_1_.png', (torch.squeeze(x)[1]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_2_.png', (torch.squeeze(x)[2]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_3_.png', (torch.squeeze(x)[3]).cpu().numpy())

        out = torch.squeeze(out)
        out = torch.unsqueeze(out, dim=0).permute(0,3,1,2)
        x = torch.squeeze(x).permute(0,3,1,2)

        # evaluate scores
        images = torch.cat([self.my_preprocess(out), self.my_preprocess(x)])
        text = torch.cat([clip.tokenize([prompt]).to(self.args.device)])
        logits_per_image, logits_per_text = self.model(images, text)

        logits = torch.squeeze(logits_per_image)
        logits = logits.reshape(1,-1)
        return logits

In [ ]:
# model
class CNN(nn.Module):
    def __init__(self, args):
        super(CNN, self).__init__()
        self.args = args
        self.prompt_path = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/data', args.mode) + '_prompts.pkl'
        with open(self.prompt_path, 'rb') as f:
          self.idx2prompt = pickle.load(f)
        print(self.idx2prompt)

        self.my_preprocess = Compose([
                                Resize(224, interpolation=InterpolationMode.BICUBIC),
                                Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
                            ])
        self.model, _ = clip.load("ViT-B/32", device=self.args.device)

        # (64,64,1) => (32,32,16) => (16,16,16) => (8,8,32) => (4,4,64)
        self.layer1 = nn.Sequential(
            nn.Conv2d(12, 16, kernel_size=6, stride=2, padding=2),
            nn.Dropout(),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=1))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=6, stride=2, padding=2),
            nn.Dropout(),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=1))
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=6, stride=2, padding=2),
            nn.Dropout(),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=1))
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=6, stride=2, padding=2),
            nn.Dropout(),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=1))
        self.fc1 = nn.Linear(12544, 128)
        self.fc2 = nn.Linear(128, 150528)

        
        self.params_to_update = list(self.layer1.parameters()) + list(self.layer2.parameters()) + list(self.layer3.parameters()) + list(self.layer4.parameters()) + list(self.fc1.parameters()) + list(self.fc2.parameters())

    def forward(self, x, prompts):
        prompts = prompts.tolist()
        prompts = [self.idx2prompt[prompt] for prompt in prompts]
        prompt = prompts[0]

        out = x.permute(0,1,4,2,3)
        out = out.reshape(1,-1,224,224)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)

        out = out.uniform_(0,1)*255
        out = out.reshape(x.size(0),1,224,224,3)

        if self.args.mode != 'train':
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'.png', (torch.squeeze(out)).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_0_.png', (torch.squeeze(x)[0]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_1_.png', (torch.squeeze(x)[1]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_2_.png', (torch.squeeze(x)[2]).cpu().numpy())
          cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/output/'+prompt+'_3_.png', (torch.squeeze(x)[3]).cpu().numpy())
        
        out = torch.squeeze(out)
        out = torch.unsqueeze(out, dim=0).permute(0,3,1,2)
        x = torch.squeeze(x).permute(0,3,1,2)

        # evaluate scores
        images = torch.cat([self.my_preprocess(out), self.my_preprocess(x)])
        text = torch.cat([clip.tokenize([prompt]).to(self.args.device)])
        logits_per_image, logits_per_text = self.model(images, text)

        logits = torch.squeeze(logits_per_image)
        logits = logits.reshape(1,-1)
        return logits

In [ ]:
# args
def get_args():
    args = easydict.EasyDict({
        "epoch": 30,
        "batch_size": 1,
        "mode": 'train',
        "ckpt": 1,
        "device": 'cuda'
    })
    return args

In [ ]:
# model run
args = get_args()
trainer = Trainer(args)

if args.mode == 'train':
  trainer.train()
else:
  trainer.validate()

data num:  183
{0: 'A spoon in a pot of fry sauce', 1: 'Animals at the guesthouse or nearby', 2: 'Art Of A Little Girl In A Blu', 3: 'Drink Like A Pirate Dance Like A Mermaid Poster', 4: 'Food at or somewhere near the hotel', 5: 'Image of Stained Concrete Flooring Throughout for The Towers Seabrook', 6: 'Lavender Flower Watercolor Yellow Save the Date Announcement', 7: 'Mind the Shoes city girl', 8: 'Nuts on a Keto Diet', 9: 'Pin By The Holiday Cottages On Wonderful Wales Cottage House', 10: 'Portrait of the young smiling sportsman stock photos', 11: 'Relaxing outside the Lamington Tea House at Binna Burra', 12: 'Sea spiders stacked on top of one another', 13: 'Teenage girl playing on a scooter stock photography', 14: 'The Big Bad Theory Throw Pillow', 15: 'Winter at the Grand Canyon Sticker', 16: 'a beautiful day with some buildings and plants ', 17: 'a blog about home and garden design including design ', 18: 'a city for sale property ', 19: 'a flower photographed from above ', 20: '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch [1/30], Step [5/183], Loss: 4.5273
Epoch [1/30], Step [10/183], Loss: 3.3926
Epoch [1/30], Step [15/183], Loss: 2.5039
Epoch [1/30], Step [20/183], Loss: 1.1992
Epoch [1/30], Step [25/183], Loss: 4.9609
Epoch [1/30], Step [30/183], Loss: 4.4414
Epoch [1/30], Step [35/183], Loss: 2.9746
Epoch [1/30], Step [40/183], Loss: 3.5996
Epoch [1/30], Step [45/183], Loss: 5.3750
Epoch [1/30], Step [50/183], Loss: 2.0371
Epoch [1/30], Step [55/183], Loss: 3.3633
Epoch [1/30], Step [60/183], Loss: 2.3730
Epoch [1/30], Step [65/183], Loss: 6.0938
Epoch [1/30], Step [70/183], Loss: 4.3320
Epoch [1/30], Step [75/183], Loss: 5.3086
Epoch [1/30], Step [80/183], Loss: 3.8789
Epoch [1/30], Step [85/183], Loss: 5.5781
Epoch [1/30], Step [90/183], Loss: 5.4102
Epoch [1/30], Step [95/183], Loss: 1.4824
Epoch [1/30], Step [100/183], Loss: 2.0215
Epoch [1/30], Step [105/183], Loss: 3.5938
Epoch [1/30], Step [110/183], Loss: 3.6523
Epoch [1/30], Step [115/183], Loss: 3.7344
Epoch [1/30], Step [120/183], L